# Вебинар 5. Домашнее задание

1) Прочитать статьи про BPR, WARP loss

2) Сделать грид серч текущей модели


LigthFM -> https://arxiv.org/pdf/1507.08439.pdf
https://making.lyst.com/lightfm/docs/home.htm

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

from utils2 import prefilter_items

/Users/coraleks/Projects/GeekBrainsAI/RecSys/venv/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [4]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/Users/coraleks/Projects/GeekBrainsAI/RecSys/homeworks/utils2.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


### 2.1 Prepare csr train matrix

In [5]:


user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2.2 Prepare CSR test matrix

In [6]:

data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
# 3. Prepare user and item features

user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(5)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
# Encoding features

user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [11]:
%%time

params = dict(loss=['bpr', 'warp'],dim_features=[ 5, 10, 15],regularization=[ 0.5, 0.05])

epochs = 10
nn=1
for loss in params['loss']:
    for dim_feature in params['dim_features']:
        for L2penalty in params['regularization']:
            model = LightFM(no_components=dim_feature,
                loss=loss,
                learning_rate=0.05, 
                item_alpha=L2penalty,
                user_alpha=L2penalty,
                random_state=42)

            model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                sample_weight=coo_matrix(user_item_matrix),
                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                epochs=epochs,
                num_threads=4,
                verbose=False)
            
            test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

            print(f'N {nn} - Loss:{loss:4}, Dimensionality of the feature:{dim_feature:2},  Regularization:{L2penalty:5}: p_at_k:{test_precision:7.5f}')
            nn += 1

N 1 - Loss:bpr , Dimensionality of the feature: 5,  Regularization:  0.5: p_at_k:0.00568
N 2 - Loss:bpr , Dimensionality of the feature: 5,  Regularization: 0.05: p_at_k:0.00284
N 3 - Loss:bpr , Dimensionality of the feature:10,  Regularization:  0.5: p_at_k:0.00069
N 4 - Loss:bpr , Dimensionality of the feature:10,  Regularization: 0.05: p_at_k:0.00284
N 5 - Loss:bpr , Dimensionality of the feature:15,  Regularization:  0.5: p_at_k:0.00206
N 6 - Loss:bpr , Dimensionality of the feature:15,  Regularization: 0.05: p_at_k:0.00274
N 7 - Loss:warp, Dimensionality of the feature: 5,  Regularization:  0.5: p_at_k:0.00049
N 8 - Loss:warp, Dimensionality of the feature: 5,  Regularization: 0.05: p_at_k:0.00225
N 9 - Loss:warp, Dimensionality of the feature:10,  Regularization:  0.5: p_at_k:0.00000
N 10 - Loss:warp, Dimensionality of the feature:10,  Regularization: 0.05: p_at_k:0.00225
N 11 - Loss:warp, Dimensionality of the feature:15,  Regularization:  0.5: p_at_k:0.00118
N 12 - Loss:warp, D

> **Оптимальные параметры модели:**
>
> Функция потерь (Loss): **bpr**
> Размерность векторов (Dimensionality of the feature): **5**
> Коэффициент регуляризация (Regularization): **0,5**